In [ ]:
from ollama import Client
from pydantic import BaseModel

class ExampleOutputClass(BaseModel):
    response : str
    evidences : list[str]

def add_two_numbers(a: int, b: int) -> int:
  """
  Add two numbers

  Args:
    a: The first integer number
    b: The second integer number

  Returns:
    int: The sum of the two numbers
  """
  return a + b

import ollama

response = ollama.chat(
  'llama3.1',
  messages=[{'role': 'user', 'content': 'What is 10 + 10?'}],
  tools=[add_two_numbers], # Actual function reference
)

# Huggingface Function Calling

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# from huggingface_hub import login
# login(token = "")

def divide(a : float, b : float):
    """
    Divides two numbers.

    Args:
        a: First number.
        b: Second number.
    """
    return a // b

tools = [divide]

model_name = "NousResearch/Hermes-2-Pro-Llama-3-8B"  # Replace with the appropriate Llama model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

c:\Users\nguyent68\Documents\GitHub\RAG-LLM\ragENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:13<00:00,  3.41s/it]
Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [2]:
def get_current_temperature(location: str):
    """
    Gets the temperature at a given location.

    Args:
        location: The location to get the temperature for, in the format "city, country"
    """
    return 22.0  # bug: Sometimes the temperature is not 22. low priority to fix tho

tools.append(get_current_temperature)

prompt = """What is the current weather in Boston?"""

content = [{"role" : "user", "content" : prompt}]


tool_prompts = tokenizer.apply_chat_template(content, tools=tools, return_tensors="pt", return_dict=True, add_generation_prompt=True).to(model.device)
outputs = model.generate(**tool_prompts, max_new_tokens=250)

# Decode the generated text
response = tokenizer.decode( outputs[0, tool_prompts['input_ids'].shape[1]:])
print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<tool_call>
{"name": "get_current_temperature", "arguments": {"location": "Boston, USA"}}
</tool_call><|im_end|>
